In [35]:
import os
from uuid import uuid4
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from tools.create_tool import create_client,create_order
from tools.query_tool import query_tool
import random

llm = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_CHAT_API_KEY"),
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_MODEL"),
    api_version=os.getenv("AZURE_OPENAI_CHAT_API_VERSION"),
    temperature=0.3,
    max_tokens=800,
    azure_endpoint=os.getenv("AZURE_OPENAI_CHAT_ENDPOINT"),
)

embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("EMBEDDING_MODEL"),
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBED_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_EMBED_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBED_API_VERSION"),
)

In [36]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./documents/studio_faq.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [39]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_dir",
    collection_name="qest_assignment"
)

In [40]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [41]:
query = "What should I expect at my first visit?"
result = qa_chain.invoke({"query": query})

print("Answer:", result["result"])
print("Sources:", [doc.metadata["source"] for doc in result["source_documents"]])


Answer: At your first visit, you can expect a warm welcome, including a studio tour during your first session, a review of any necessary paperwork, and a brief orientation on the class format and etiquette.
Sources: ['./documents/studio_faq.pdf', './documents/studio_faq.pdf', './documents/studio_faq.pdf', './documents/studio_faq.pdf', './documents/studio_faq.pdf']
